# Likelihood Optimization of gas Kinematics in IFUs (LOKI)
## Fitting example: NIRSpec multichannel model (aperture)

Michael Reefe

This example notebook provides a tutorial on how to run LOKI on a multi-channel NIRSpec IFU cube, combining data from the F100LP, F170LP, and F290LP filters, and fitting an integrated spectrum within an aperture.

In [1]:
using Pkg
Pkg.activate(dirname(@__DIR__))
Pkg.instantiate()
Pkg.precompile()
using Loki
using Unitful 

  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`


Now we want to load in our data. For this example, we'll be using the data for NGC 7469 in three filter/grating combinations, which are located in the same folder as this notebook. The JWST reduced data does not include a redshift, so we must provide the redshift ourselves.  We can use the `from_fits` function to load in the JWST-formatted FITS files, along with the redshift.

In [2]:
# The redshift of the target object: NGC 7469
z = 0.016317
# The semicolon at the end suppresses printing the output Observation object, which is long and not very enlightening
obs = from_fits(["f100lp_g140h-f100lp_s3d.fits.gz", "f170lp_g235h-f170lp_s3d.fits.gz", "f290lp_g395h-f290lp_s3d.fits.gz"], z);

[ Info: Initializing DataCube struct from f100lp_g140h-f100lp_s3d.fits.gz
[ Info: Initializing DataCube struct from f170lp_g235h-f170lp_s3d.fits.gz
[ Info: Initializing DataCube struct from f290lp_g395h-f290lp_s3d.fits.gz


Next, we create some variables that we will use later. We will be fitting data from multiple grating/filter combos, and we can take the `name` property from the Observation object we just loaded in to get the name of the target. Here, `run_name` is just a unique identifier that we will use for this run.

In [3]:
channel = "NIRSpecAll"
nm = replace(obs.name, " " => "_") 
run_name = "$(nm)_$(channel)_nuc_aperture"

"NGC_7469_NIRSpecAll_nuc_aperture"

Before fitting, we want to do some pre-processing on the data. We want to convert the data to the rest-frame, mast out / interpolate any bad pixels, and replace the JWST pipeline-generated errors with some more realistic ones.  All of this is achieved in the next block of code.

In [4]:
if isfile("$nm.channel$channel.rest_frame.fits")
    # If we've already performed this step in a previous run, just load in the pre-processed data
    obs = from_fits(["$nm.channel$channel.rest_frame.fits"], obs.z);
    
else
    # Convert to rest-frame wavelength vector, and mask out bad spaxels
    correct!(obs)

    channels = [:G140H_F100LP, :G235H_F170LP, :G395H_F290LP]
    # Reproject the sub-channels onto the same WCS grid and combine them into one full channel
    # - The channels vector gives the names of each channel to concatenate. By default, JWST subchannels are
    #   given labels of "A" for short, "B" for medium, and "C" for long, followed by the channel number.  
    # - The "out_id" argument will determine the label given to the combined channel data. 
    combine_channels!(obs, channels, out_id=channel, order=1, adjust_wcs_headerinfo=true, extract_from_ap=0.)

    # rotate to the RA/Dec axes on the sky
    rotate_to_sky_axes!(obs.channels[channel])

    # We interpolate any rogue NaNs using a linear interpolation, since the MPFIT minimizer does not handle NaNs well.
    interpolate_nans!(obs.channels[channel])

    # Finally, we calculate the statistical errors (i.e. the standard deviation of the residuals with a cubic spline fit)
    # and replace the errors in the cube with these, since the provided errors are typically underestimated.
    # You can skip this step if you wish to use the default errors.
    calculate_statistical_errors!(obs.channels[channel])
    
    # Save the pre-processed data as a FITS file so it can be quickly reloaded later
    save_fits(".", obs, [channel]);
end


new_wave contains values outside the range in old_wave, new_fluxes and new_errs will be filled with the value set in the 'fill' keyword argument. 


new_wave contains values outside the range in old_wave, new_fluxes and new_errs will be filled with the value set in the 'fill' keyword argument. 

[ Info: Using SFD98 dust map at (α=345.8150416666667°, δ=8.873894444444446°): E(B-V)=0.06873496901798093
[ Info: Using SFD98 dust map at (α=345.8150416666667°, δ=8.873894444444446°): E(B-V)=0.06873496901798093
[ Info: Using SFD98 dust map at (α=345.8150416666667°, δ=8.873894444444446°): E(B-V)=0.06873496901798093
[ Info: Aligning World Coordinate Systems for channels [:G140H_F100LP, :G235H_F170LP, :G395H_F290LP]...
[ Info: The centroid offset relative to channel G140H_F100LP for channel G140H_F100LP is (0, 0) spaxels
[ Info: The centroid offset relative to channel G140H_F100LP for channel G235H_F170LP is (-0.039, -0.22) spaxels
[ Info: The centroid offset relative to channel G140H_F100LP for c

Progress: 100%|█████████████████████████████████████████| Time: 0:02:16


[ Info: Reprojecting channel F290LP G395H onto the optimal (51, 45) WCS grid...


Progress: 100%|█████████████████████████████████████████| Time: 0:02:14



new_wave contains values outside the range in old_wave, new_fluxes and new_errs will be filled with the value set in the 'fill' keyword argument. 

┌ Warning: The wavelength dimension has not be resampled to be linear when concatenating multiple full channels! Only overlapping regions between channels have been resampled to a median resolution!
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/core/cubedata.jl:2468
[ Info: Masking bins with bad data...
[ Info: Redoing logarithmic wavelength binning...
[ Info: Done!
[ Info: Interpolating NaNs in cube with channel MULTIPLE, band MULTIPLE:
[ Info: Calculating statistical errors for each spaxel...


Progress:  90%|████████████████████████████████████▊    |  ETA: 0:00:46Excessive output truncated after 524332 bytes.

We next create an aperture to define the region of interest that we would like to fit. We can do this with the `make_aperture` function. We can customize the aperture's shape, centroid, radius, etc.

In [5]:
# - The first argument is the data cube
# - The second argument is the aperture shape, which may be one of: (Circular, Rectangular, Elliptical)
# - Next are the right ascension in sexagesimal hours and the declination in sexagesimal degrees
# - The next arguments depend on the aperture shape:
#    - For circles, it is the radius in arcseconds
#    - For rectangles, it is the width in arcseconds, height in arcseconds, and rotation angle in degrees
#    - For ellipses, it is the semimajor axis in arcseconds, semiminor axis in arcseconds, and rotation angle in degrees
# - The auto_centroid argument, if true, will adjust the aperture centroid to the closest peak in brightness
# - The scale_psf argument, if true, will create a series of apertures (as a function of wavelength) with increasing radii 
#   that scale at the same rate as the PSF
ap = make_aperture(obs.channels[channel], :Circular, "23:03:15.610", "+8:52:26.10", 0.5, auto_centroid=true)

11×11 Photometry.Aperture.CircularAperture{Float64} with indices 21:31×27:37:
 0.0          0.0         0.0       …  0.0232627  0.0       0.0
 0.0          0.00746973  0.581917     0.820831   0.112055  0.0
 0.0          0.479336    1.0          1.0        0.808096  0.0159328
 0.0452131    0.950514    1.0          1.0        1.0       0.34042
 0.260201     1.0         1.0          1.0        1.0       0.604894
 0.313734     1.0         1.0       …  1.0        1.0       0.658427
 0.163444     1.0         1.0          1.0        1.0       0.508136
 0.000378579  0.788039    1.0          1.0        0.983891  0.149219
 0.0          0.17843     0.942279     0.999993   0.465409  0.0
 0.0          0.0         0.168787     0.37325    0.0       0.0
 0.0          0.0         0.0       …  0.0        0.0       0.0

Finally, we create the `CubeFitter` object and call the `fit_cube!` function to fit the data. Note specifically for NIRSpec data, we want to enable the option "nirspec_mask_chip_gaps".  This will make sure that when fitting any integrated regions larger than a single spaxel (i.e. apertures, voronoi bins, or the initial integrated fit over the whole FOV) the chip gap regions will be fully masked out.  This is necessary because the wavelengths of the chip gaps vary based on the position in the IFU, so when making integrated spectra, in regions where some spaxels are masked and others aren't, it can create artifical dips or peaks in the continuum.  The "nirspec_mask_chip_gaps" options masks out the maximum extent of the chip gaps over all of the IFU positions (but again, only when fitting regions larger than a single spaxel).

\*\*\*IMPORTANT\*\*\* For this example to look like the plot I've shown at the bottom, you'll need to enable a power law component to the continuum fit. You can do so by editing the "optical.toml" config file and uncommenting the power law section (specifically the entry for "[[power_law_indices]]"). Then run the cell below to fit with the power law component.  You'll also need to edit the "lines.toml" config file to enable a second velocity component for all kinematic groups.  This can be done by uncommenting the lines under the "[acomps]" key near the very bottom of the file.

In [6]:
# To see a full list of keyword arguments, please refer to the docstring, which can be accessed by typing `?CubeFitter` in the command
# line after importing Loki.
cube_fitter = CubeFitter(
    obs.channels[channel], 
    obs.z, 
    run_name; 
    nirspec_mask_chip_gaps=true,
    parallel=false, 
    plot_spaxels=:both, 
    plot_maps=true, 
    save_fits=true,
    silicate_absorption="d+",
    extinction_screen=true, 
    use_pah_templates=true,
    fit_sil_emission=true, 
    fit_stellar_continuum=false, 
    save_full_model=true, 
    map_snr_thresh=3., 
    subtract_cubic_spline=true,
    # we will use a custom emission line mask for this one because the lines vary in width quite a bit,
    # so the default behavior of masking +/-500 km/s from each line position doesn't work very well
    linemask_overrides=[
        (0.995, 1.020) .* u"μm",
        (1.069, 1.107) .* u"μm",
        (1.120, 1.140) .* u"μm",
        (1.273, 1.294) .* u"μm",
        (1.857, 1.900) .* u"μm",
        (1.933, 1.967) .* u"μm",
        (2.153, 2.183) .* u"μm",
        (2.609, 2.640) .* u"μm",
        (3.014, 3.043) .* u"μm",
        (3.292, 3.303) .* u"μm",
        (3.733, 3.762) .* u"μm",
        (4.462, 4.500) .* u"μm",
        (4.512, 4.540) .* u"μm",
        (4.641, 4.704) .* u"μm",
        (5.047, 5.056) .* u"μm",
        (5.135, 5.145) .* u"μm"
    ],
    # we will also do a line test to see if a 2nd velocity component is statistically significant; here 
    # we choose which lines will be tested for each kinematic group
    line_test_lines=[["HeI_1083m", "HI_Pa_alpha"], ["ArVI_4529m"], ["MgVII_3027m"]]
)

# Call the fit_cube! function on the cube_fitter object, using the aperture we defined.
fit_cube!(cube_fitter, ap)

(CubeFitter{Float64, Int64, Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, Quantity{Float64, 𝐋 𝐓⁻¹, Unitful.FreeUnits{(km, s⁻¹), 𝐋 𝐓⁻¹, nothing}}, Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}(DataCube{Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}, Array{Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, 3}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}[0.9547734895800328 μm, 0.9550047726596355 μm, 0.9552361117649484 μm, 0.9554675069095432 μm, 0.9556989581069948 μm, 0.9559304653708812 μm, 0.9561620287147841 μm, 0.9563936481522881 μm, 0.9566253236969813 μm, 0.956857055362455 μm  …  5.173831522439116 μm, 5.1750848246106935 μm, 5.1763384303805475 μm, 5.17759233982222 μm, 5.178846553009275 μm, 5.180101070015289 μm, 5.18135589091386 μm, 5.182611015778601 μm, 5.1838664446831455 μm, 5.185122177701144 μm], Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻

And the results can be found in the "output_[run_name]" directory, just like the other examples!  Here is what the fit looks like:

![](./NGC7469.NIRSpecAll.aperture.png)

The orange line shows the final model.  The decomposed components of the model consist of:
- Thermal dust continuum and a power law, in gray
- Hot dust emission, in light green
- Emission lines, in purple
- Extinction, in dotted gray (read from the right axis)

The gray band(s) shows a masked out part of the spectrum -- this is due to NIRSpec's chip gaps.  There are three of them because this is combined data from 3 filters.

We can also see the results of our line tests that determined whether multiple components were necessary.  As an example, here is Paschen alpha:

![](./NGC7469.NIRSpecAll.linetest.png)
